In [1]:
!nvidia-smi

Sun Dec 31 15:50:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:07:00.0  On |                  N/A |
| 38%   39C    P8    25W / 215W |    478MiB /  8192MiB |     33%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os

import cupy as cp
import pandas as pd

import dask_cudf
import cudf

cp.random.seed(12)

In [3]:
s = cudf.Series([1, 2, 3, None, 4])
s

0       1
1       2
2       3
3    <NA>
4       4
dtype: int64

In [4]:
df = cudf.DataFrame(
    {
        "a": list(range(20)),
        "b": list(reversed(range(20))),
        "c": list(range(20)),
    }
)
df

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2
3,3,16,3
4,4,15,4
5,5,14,5
6,6,13,6
7,7,12,7
8,8,11,8
9,9,10,9


In [5]:
ddf = dask_cudf.from_cudf(df, npartitions=2)
ddf.head()

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2
3,3,16,3
4,4,15,4


In [6]:
pdf = pd.DataFrame({"a": [0, 1, 2, 3], "b": [0.1, 0.2, None, 0.3]})
gdf = cudf.DataFrame.from_pandas(pdf)
gdf

,a,b
0,0,0.1
1,1,0.2
2,2,<NA>
3,3,0.3


In [7]:
dask_gdf = dask_cudf.from_cudf(gdf, npartitions=2)
dask_gdf.head(n=2)

,a,b
0,0,0.1
1,1,0.2


In [8]:
df.sort_values(by="b")

,a,b,c
19,19,0,19
18,18,1,18
17,17,2,17
16,16,3,16
15,15,4,15
14,14,5,14
13,13,6,13
12,12,7,12
11,11,8,11
10,10,9,10


In [10]:
ddf.sort_values(by="b").head()

,a,b,c
19,19,0,19
18,18,1,18
17,17,2,17
16,16,3,16
15,15,4,15


In [9]:
df["a"]

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
Name: a, dtype: int64

In [11]:
df.loc[2:5, ["a", "b"]]

,a,b
2,2,17
3,3,16
4,4,15
5,5,14


In [12]:
ddf.loc[2:5, ["a", "b"]].head()

,a,b
2,2,17
3,3,16
4,4,15
5,5,14


In [13]:
df.iloc[0]

a     0
b    19
c     0
Name: 0, dtype: int64

In [14]:
df.iloc[0:3, 0:2]

,a,b
0,0,19
1,1,18
2,2,17


In [15]:
df[3:5]

,a,b,c
3,3,16,3
4,4,15,4


In [16]:
df[df.b > 15]

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2
3,3,16,3


In [17]:
ddf[ddf.b > 15].head(n=3)

,a,b,c
0,0,19,0
1,1,18,1
2,2,17,2


In [18]:
df.query("b == 3")

,a,b,c
16,16,3,16


In [19]:
ddf.query("b == 3").compute()

,a,b,c
16,16,3,16


In [20]:
cudf_comparator = 3
df.query("b == @cudf_comparator")

,a,b,c
16,16,3,16


In [21]:
dask_cudf_comparator = 3
ddf.query("b == @val", local_dict={"val": dask_cudf_comparator}).compute()

,a,b,c
16,16,3,16


In [22]:
df[df.a.isin([0, 5])]

,a,b,c
0,0,19,0
5,5,14,5
